In [44]:
import random
import numpy as np
import os
from os.path import join
import torch
from multiprocessing.dummy import Pool
from tqdm import tqdm
import cv2

In [45]:
import matplotlib.pyplot as plt

In [46]:
scale = 4
dataset = "CUFDE5"
in_dir = dataset+"_RAW"
hr_dir = dataset+"_HR"
lr_dir = dataset+f"_LR_x{scale}" 
ref_dir = dataset+f"_Ref_x{scale}" 
test = False
mode="REGULAR"

Function

In [47]:
def show(img):
    plt.imshow(cv2.cvtColor(img,cv2.COLOR_BGR2RGB))

In [48]:
def downscale_LR(image,scale):
	heightHR, widthHR = image.shape[:2]
	heightLR, widthLR = heightHR // scale , widthHR // scale 
	return cv2.resize(image,(widthLR,heightLR), interpolation=cv2.INTER_CUBIC)

In [49]:
def crop_ref(image, scale,offset=(0,0)):
	heightHR, widthHR = image.shape[:2]
	heightRef, widthRef = heightHR // scale , widthHR // scale 
	downRef, rightRef = offset
	crop_coord = [downRef, downRef+heightRef, rightRef, rightRef+widthRef]
	return image[downRef:downRef+heightRef, rightRef:rightRef+widthRef], crop_coord


In [50]:
def crop_multi_ref(image, scale, mode="REGULAR"):
	heightHR, widthHR = image.shape[:2]
	heightRef, widthRef = heightHR // scale , widthHR // scale 
	offset = []
	if mode=="REGULAR":
		offsets = [
			((heightHR-heightRef) // 2,(widthHR-widthRef) // 2),
			((heightHR-heightRef) // 4,(widthHR-widthRef) // 4),
			((heightHR-heightRef) // 4,(widthHR-widthRef)*3 // 4),
			((heightHR-heightRef)*3 // 4,(widthHR-widthRef) // 4),
			((heightHR-heightRef)*3 // 4,(widthHR-widthRef)*3 // 4),
		]
	elif mode=="EXTREME":
		offsets = [
			((heightHR-heightRef) // 2,(widthHR-widthRef) // 2),
			(0,0),
			(0,widthHR-widthRef),
			(heightHR-heightRef,0),
			(heightHR-heightRef,widthHR-widthRef),
		]
	output = [crop_ref(image, scale,offset) for offset in offsets]
	return output

Main

In [51]:
os.makedirs(ref_dir, exist_ok = True) 
os.makedirs(lr_dir, exist_ok = True) 
os.makedirs(hr_dir, exist_ok = True) 

for input in os.listdir(in_dir):  
    name, ext = os.path.splitext(input)

    input_full = os.path.join(in_dir, name+ext )
    hr = cv2.imread(input_full)
    assert hr is not None

    refs = crop_multi_ref(hr,scale,mode)    
    for idx, ref in enumerate(refs):
        ref, _ = ref
        ref_full = os.path.join(ref_dir, f"{name}-{idx}{ext}")
        assert cv2.imwrite(ref_full, ref)

        lr_full = os.path.join(lr_dir, f"{name}-{idx}{ext}")
        lr = downscale_LR(hr,scale)
        assert cv2.imwrite(lr_full, lr)

        hr_full = os.path.join(hr_dir, f"{name}-{idx}{ext}")
        assert cv2.imwrite(hr_full, hr)